In [1]:
# Primary author: Johannes (a.k.a. James) P. Johnson-Martinez
# DOI: https://doi.org/10.1101/2023.03.04.531100
# Title: base_final
# Input(s): Arivale data snapshots, metadata.csv, asvs.csv
# Output(s): Statistical graphs and conclusions,
# chemistries.csv, chemistries_count.csv,
# clrtaxa.csv, eGFR.csv,
# gutmicrobiome.csv, gutmicrobiome_count.csv,
# metabolomics.csv, metabolomics_count.csv, metabolomics_fullmetadata.csv,
# proteomics.csv, proteomics_count.csv, proteomics_metadata_table.csv
# taxa.csv, ordinal_questions.csv,
# depression.csv, anxiety.csv,
# nonPTR_cohort.csv
# Comments: 
# This is the primary notebook that obtains the several different data snapshots, 
# transforms and arranges the data, and outputs the .csv files that are inputted into
# the R markdown files (.rmd) to compute final graphs and statistics

# Initial steps:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns

# Get the library
from arivale_data_interface import *
import arivale_data_interface as adi

# Set display options
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.display import display

# Apply these settings so the library knows where the data is
adi.apply_isb_config()

# If you want plots to show up in the notebook you need to run this
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 10]

In [2]:
#list the local snapshots
local_snapshots = adi.list_local_snapshots()
with pd.option_context('display.max_colwidth', 100):
    display(local_snapshots)
adi.list_snapshot_contents(path = local_snapshots.loc[0])

0     /proj/arivale/snapshots/arivale_snapshot_ISB_2020-03-16_2156
1     /proj/arivale/snapshots/arivale_snapshot_ISB_2020-01-24_0127
2     /proj/arivale/snapshots/arivale_snapshot_ISB_2019-11-29_2258
3     /proj/arivale/snapshots/arivale_snapshot_ISB_2019-11-22_2258
4     /proj/arivale/snapshots/arivale_snapshot_ISB_2019-10-14_0054
5     /proj/arivale/snapshots/arivale_snapshot_ISB_2019-05-31_2326
6     /proj/arivale/snapshots/arivale_snapshot_ISB_2019-05-19_1330
7     /proj/arivale/snapshots/arivale_snapshot_ISB_2019-05-12_1330
8     /proj/arivale/snapshots/arivale_snapshot_ISB_2019-05-10_0053
9     /proj/arivale/snapshots/arivale_snapshot_ISB_2019-05-05_1330
10    /proj/arivale/snapshots/arivale_snapshot_ISB_2019-04-28_1330
11    /proj/arivale/snapshots/arivale_snapshot_ISB_2019-04-21_1330
12    /proj/arivale/snapshots/arivale_snapshot_ISB_2019-04-14_1330
13    /proj/arivale/snapshots/arivale_snapshot_ISB_2019-03-31_1330
14    /proj/arivale/snapshots/arivale_snapshot_ISB_2019-03-29_

,basename,dirname,size,isdir,mode,mtime
0,proteomics_metadata.tsv,/proj/arivale/snapshots/arivale_snapshot_ISB_2...,248162,False,33184,2020-03-16 21:59:20.000000000
1,microbiome_dashboard.tsv,/proj/arivale/snapshots/arivale_snapshot_ISB_2...,1389505,False,33184,2020-03-16 21:58:32.000000000
2,assessments_exercise_readiness.tsv,/proj/arivale/snapshots/arivale_snapshot_ISB_2...,552055,False,33184,2020-03-16 21:57:17.000000000
3,assessments_health_history_old.tsv,/proj/arivale/snapshots/arivale_snapshot_ISB_2...,30775811,False,33184,2020-03-16 21:57:15.000000000
4,analyte_ranges.tsv,/proj/arivale/snapshots/arivale_snapshot_ISB_2...,612873,False,33184,2020-03-16 21:57:01.000000000
5,wearables_raw_monthly.tsv,/proj/arivale/snapshots/arivale_snapshot_ISB_2...,27708310,False,33184,2020-03-16 22:00:26.000000000
6,metabolomics_corrected.tsv,/proj/arivale/snapshots/arivale_snapshot_ISB_2...,60169491,False,33184,2020-03-16 21:59:26.000000000
7,assessments.tsv,/proj/arivale/snapshots/arivale_snapshot_ISB_2...,182233974,False,33184,2020-03-16 21:57:09.000000000
8,microbiome_trimmed_diversity.tsv,/proj/arivale/snapshots/arivale_snapshot_ISB_2...,1657995,False,33184,2020-03-16 21:59:11.000000000
9,assessments_lifestyle.tsv,/proj/arivale/snapshots/arivale_snapshot_ISB_2...,3794590,False,33184,2020-03-16 21:57:17.000000000


In [3]:
#Get the principal components of genetic ancestry
anc = adi.get_snapshot('genetics_ancestry')
anc = anc[['public_client_id','PC1','PC2','PC3','PC4','PC5']]
#anc = anc.drop('days_in_program', axis=1)
anc = anc.drop_duplicates(subset='public_client_id')
anc
#4761 participants

,public_client_id,PC1,PC2,PC3,PC4,PC5
0,01000261,0.006041,-0.001819,-0.001843,0.005782,0.002251
1,01001298,0.006588,-0.002195,-0.004136,0.009007,0.004797
3,01001467,-0.024357,0.065986,-0.013913,0.003081,0.006945
4,01001548,0.006395,-0.002255,-0.003103,0.008462,0.002178
5,01001621,0.006223,-0.001662,-0.003846,0.006116,0.001820
...,...,...,...,...,...,...
4757,QIRL,0.006607,-0.002364,-0.004081,0.007232,0.003434
4758,ZU273983,0.006136,-0.001745,-0.005167,0.007240,0.001996
4759,ZU612255,-0.026077,0.087718,-0.016342,0.004520,0.005992
4760,ZU621944,0.002336,0.003509,0.018513,-0.034476,-0.000915


In [4]:
# Blood plasma metabolites snapshot:
# Get the metabolomics data:
rawmet=adi.get_snapshot('metabolomics_corrected')
rawmet = rawmet.sort_values('days_in_program',ascending=True) # Sort by days
rawmet = rawmet.sort_values('public_client_id',ascending=True) # Sort by PCI
rawmet=rawmet.drop(columns={"month","weekday","season"}) # Drop unneeded columns
#rawmet = rawmet.drop_duplicates('public_client_id') # This line drops duplicates from this snapshot from 3305 participants -> 2043
met_nonnancounts=(~rawmet.isna()).sum() # sum up times any columns have non-NA values
rawmet = rawmet.loc[:, met_nonnancounts/rawmet.shape[0] >= 0.70] #this sum of non-NA counts/num of total rows should >= 70% to proceed with the cohort
rawmet = rawmet.dropna(axis = 0, how = 'all') # Drop patients with all NaN values
rawmet = rawmet.dropna(axis = 1, how = 'all') # Drop patients with all NaN values
rawmet.iloc[:,5:] = rawmet.iloc[:,5:].fillna(rawmet.iloc[:,5:].median()) # Fill df NaNs with median instead of 0
#rawmet = rawmet[['public_client_id','days_in_program','100001315','100015967','100001417','100006191'
              #  ,'999952286'
              #  ,'999946354'
             ##   ,'999946486'
              #  ,'999946510'
              #  ,'999946616'
              #  ,'999949883'
              #  ,'999946510'
              #  ,'999946620']]
#rawmet = rawmet.rename(columns={'100001315':'PCS','100015967':'CD','100001417':'PAG','100006191':'PCG'})
rawmet
#3305 participants

,public_client_id,sample_id,days_in_program,days_since_first_call,days_since_first_draw,35,50,55,62,93,98,111,112,132,136,158,171,179,180,181,182,189,192,194,197,209,212,229,231,234,235,240,241,244,250,252,254,266,267,273,275,278,279,297,302,310,313,327,330,331,338,339,340,342,344,355,356,358,361,363,376,391,397,407,409,415,424,432,439,444,445,452,460,461,480,482,491,498,501,503,504,512,519,533,535,537,561,563,564,565,566,572,796,798,799,800,803,806,811,815,...,999946595,999946601,999946602,999946607,999946608,999946610,999946613,999946616,999946620,999946623,999946624,999946627,999946632,999946633,999946636,999946639,999946645,999946646,999946657,999946666,999946673,999946674,999946681,999946685,999946690,999946695,999946700,999946701,999946707,999946710,999946902,999946905,999946911,999946966,999946970,999946972,999946973,999946977,999946986,999946997,999946998,999947006,999947011,999947417,999947437,999947493,999947642,999947650,999947654,999947670,999947671,999947687,999947708,999947788,999947802,999947804,999947818,999947820,999947905,999947929,999947955,999947959,999947971,999947977,999947988,999947993,999948001,999948009,999948047,999948076,999948081,999948094,999949463,999949512,999949515,999949517,999949521,999949555,999949557,999949592,999949637,999949679,999949681,999949883,999952025,999952286,999952483,999952501,999952502,999952524,999952533,999952640,999952843,999952865,999952867,999952877,999952909,999953157,999953267,999954840
0,01000261,A477AV558-002,65,-9.0,0,0.972222,2.568898,1.155771,0.824435,0.947589,0.772955,0.716831,1.193115,5.372793,24.532961,1.289964,1.135706,0.815813,0.710183,0.986860,0.603223,1.421159,1.128161,1.186826,1.186918,2.183882,1.129249,0.920111,1.374374,1.891255,0.819227,1.204927,1.203517,1.017136,0.921926,1.530622,0.389302,1.364602,1.437235,1.702655,0.954904,2.320103,0.708487,4.268696,2.899063,0.816147,3.386740,1.340840,1.101887,0.785114,0.943850,0.875629,1.076899,1.014633,1.051367,1.239469,2.139945,1.987275,3.027573,1.165933,1.031538,1.053787,1.165150,1.364448,1.219670,1.115540,0.832081,1.370398,0.920689,1.555217,1.408812,0.800073,1.035181,0.934017,1.245427,1.512853,0.992184,1.512646,4.848959,1.096566,2.289297,1.570808,1.039696,0.889427,0.980195,1.237122,0.948468,1.329892,1.461135,1.029702,1.077272,0.924483,0.496321,1.188124,0.933092,1.196293,0.796418,0.744728,1.375578,0.748462,...,0.970412,1.734915,0.512034,2.204863,0.591331,1.770225,1.148889,2.695183,0.819572,0.606842,1.380649,5.777018,1.152754,1.220782,0.167234,0.506707,1.376055,0.585078,0.733665,15.507542,7.548603,1.002886,1.449365,0.960719,1.252370,0.953545,1.799173,2.910742,0.651738,0.378580,0.571384,1.854875,2.604398,1.030214,1.039960,1.128947,0.927640,0.924388,1.142827,0.730759,4.020639,0.993381,2.206431,0.885257,1.115327,1.835164,0.588192,0.298005,3.765115,1.168743,0.467600,0.918908,2.920539,1.087581,0.739217,0.754325,0.462004,0.689655,1.012121,0.994050,0.891436,1.069549,0.671550,0.446172,0.809453,1.266994,4.729395,1.144651,0.790954,0.984952,1.053664,1.000000,0.546438,1.206396,1.436551,1.001327,1.585908,0.483140,0.494334,0.689787,1.214707,1.310367,0.523531,0.570142,0.878512,1.266454,0.934197,1.365642,1.165906,0.999463,1.371351,1.089485,0.856545,1.678183,1.144330,0.715138,1.114859,0.592175,0.923405,0.987336
2,01001621,A776BI445-003,11,-31.0,0,2.540378,0.568236,1.328050,0.977196,1.366908,0.967574,1.957150,0.928533,0.456719,1.732912,1.103766,0.893238,0.985511,0.995848,1.504583,0.851745,1.003719,0.802254,1.359346,0.408849,1.213243,1.080032,0.844420,1.254205,0.603340,1.733442,1.165452,1.508164,1.460673,0.553575,1.147789,1.155400,0.980912,1.052493,0.971487,1.134848,0.474432,1.844409,0.664441,0.983395,1.074128,0.661147,0.387048,1.958801,0.443924,1.060663,7.146501,1.029642,0.933483,1.562872,0.792702,0.905864,0.981730,0.806218,1.049168,1.026207,1.420774,0.874352,0.797111,1.097956,0.981784,0.825712,0.795934,0.978952,0.829953,0.759106,0.824048,0.923527,0.800240,1.007761,0.730528,0.992184,0.801623,1.313013,1.041510,1.007206,1.223909,0.999185,

In [ ]:
#Output the metabolomics metatadata df from Arivale
meta = adi.get_snapshot('metabolomics_metadata')
meta.to_csv('metabolomics_fullmetadata.csv')

In [5]:
#Obtain Bowel Movement Frequency (BMF) data:
bowel = adi.get_snapshot('assessments')
bowel = bowel.sort_values('days_in_program',ascending=True) # Sort df by days first
bowel = bowel.drop(columns = ['vendor','days_since_first_call','days_since_first_draw']) # Drop days and vendor, unneeded from this point
bowel = bowel.drop_duplicates(subset='public_client_id') # Keep only the first timepoint for each public_client_id
bowel = bowel.fillna(0) # Fill in empty BMF data with zero
bowel = bowel.set_index('public_client_id') # Arrange df by index of PCI
bowel = bowel[['assessment:digestion:bowel-movements:enum','days_in_program']]
bowel = bowel.rename(columns={"assessment:digestion:bowel-movements:enum":"bowel"}) # rename column to "bowel"
bowel = bowel[bowel['bowel'] != 0] #remove participants with no bowel data, reduce from 5764 -> 3955 individuals
bowel_list = ['(1) 2 or fewer times per week', '(2) 3-6 times per week', '(3) 1-3 times daily', '(4) 4+ times daily']
bowel_r = [1,2,3,4] #convert bowel responses to ordinal bowel movement frequency categories
bowel.bowel = bowel.bowel.replace(to_replace = bowel_list, value = bowel_r)
bowel = bowel.reset_index()
bowel = bowel.drop('days_in_program', axis=1)
bowel
#3955 participants

,public_client_id,bowel
0,01092042,3
1,01568974,3
2,01684342,2
3,01601230,3
4,01124029,3
...,...,...
3950,01395286,3
3951,01232614,3
3952,01181682,3
3953,01380110,3


In [6]:
#Obtain "eGFR" calculated GFR data from the blood clinical laboratory chemistries data (credit: Alexandra Ralevski) code chunk below:
################################################################################################
chems = adi.get_snapshot('chemistries', clean=True)
cl = adi.get_snapshot('clients', clean=True)
chems_dems = join_clients(cl, chems)
#these are the relevant column names
[col for col in chems.columns if 'CREAT' in col]
[col for col in chems.columns if 'GFR' in col]
def f(sex, age, creatinine): 
    if sex == 'F':
        if (creatinine/0.7) <=1:
            return 142*(((creatinine/0.7)**-0.241) * (0.9938**age) * 1.012)
        else:
            return 142*(((creatinine/0.7)**-1.200) * (0.9938**age) * 1.012)
    if sex == 'M':
        if (creatinine/0.9) <=1:
            return 142*(((creatinine/0.9)**-0.302) * (0.9938**age))
        else:
            return 142*(((creatinine/0.7)**-1.200) * (0.9938**age))

creat_calc = chems_dems[['public_client_id','days_in_program','sex', 'age', 'CREATININE ENZ, SER']]

creat_calc['eGFR'] = creat_calc.apply(lambda x: f(x.sex, x.age, x['CREATININE ENZ, SER']), axis=1)
creat_calc.to_csv('eGFR.csv',index = False)
################################################################################################

#prepare eGFR dataframe:
eGFR = creat_calc
eGFR = eGFR.drop_duplicates(subset='public_client_id')
eGFR = eGFR.set_index('public_client_id')
eGFR = eGFR[['eGFR','days_in_program']] #select relevant columns
#drop duplicate entries to reduce 12316 entries across all individuals
#to 6133 individuals
eGFR = eGFR.reset_index()
eGFR = eGFR.drop('days_in_program', axis=1)
eGFR
#6133

/tmp/ipykernel_663465/2578788160.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  creat_calc['eGFR'] = creat_calc.apply(lambda x: f(x.sex, x.age, x['CREATININE ENZ, SER']), axis=1)


,public_client_id,eGFR
0,01000552,NaN
1,01000978,NaN
2,01001181,NaN
3,01001298,99.024207
4,01001467,100.959522
...,...,...
6128,QIRL,111.687296
6129,ZU273983,118.179626
6130,ZU612255,70.256126
6131,ZU621944,120.472213


In [7]:
#Get the clinical laboratory chemistries data:
chem=adi.get_snapshot('chemistries')
chemdf = chem.sort_values(by='days_in_program') #keep only first date data
chemdf = chemdf.drop_duplicates(subset='public_client_id')
dip = chemdf[['public_client_id','days_in_program']]
crp = chemdf[['public_client_id','days_in_program','CRP HIGH SENSITIVITY']]
ldl = chemdf[['public_client_id','days_in_program','LDL-CHOL CALCULATION']]
a1c = chemdf[['public_client_id','days_in_program','GLYCOHEMOGLOBIN A1C']]
chemdf = chemdf.set_index('public_client_id')
chemdf = chemdf.iloc[:,12:] #keep only relevant columns
chem_nonnancounts=(~chemdf.isna()).sum() # sum up times any columns have non-NA values
chemdf = chemdf.loc[:, chem_nonnancounts/chemdf.shape[0] >= .70] #this sum of non-NA count values/num of total rows should >= 70% to proceed with the cohort
chemdf = chemdf.fillna(chemdf.median()) # impute with the median
chemdf = chemdf.dropna()
chemdf = chemdf.reset_index()
chemdf = pd.merge(dip,chemdf,on=['public_client_id'])
chemdf = chemdf.drop('days_in_program', axis=1)
chemdf 
#4881 participants

,public_client_id,"ADIPONECTIN, SERUM",ALAT (SGPT),ALBUMIN,ALKALINE PHOSPHATASE,ARACHIDONIC ACID,ASAT (SGOT),BASOPHILS,BASOPHILS ABSOLUTE,"BILIRUBIN, TOTAL",BUN/CREAT RATIO,CALCIUM,CARBON DIOXIDE (CO2),CHLORIDE,"CHOLESTEROL, TOTAL","CREATININE ENZ, SER",CRP HIGH SENSITIVITY,DHA,DPA,EOSINOPHILS,EOSINOPHILS ABSOLUTE,EPA,FERRITIN,"GFR, MDRD","GFR, MDRD, AFRICAN AM",GGT,GLOBULIN,GLUCOSE,GLYCOHEMOGLOBIN A1C,HDL CHOL DIRECT,HDL PARTICLE NUMBER,HEMATOCRIT,HEMOGLOBIN,HOMA-IR,"HOMOCYSTEINE, SERUM",IMMATURE GRANULOCYTES,IMMATURE GRANULOCYTES ABSOLUTE,INSULIN,LDL PARTICLE NUMBER,LDL SMALL,LDL-CHOL CALCULATION,LDL_SIZE,LINOLEIC_ACID,LPIR_SCORE,LYMPHOCYTES,LYMPHOCYTES ABSOLUTE,"MAGNESIUM, SERUM",MCH,MCHC,MCV,"MERCURY, BLOOD",METHYLMALONIC ACID,MONOCYTES,MONOCYTES ABSOLUTE,OMEGA-3 INDEX,OMEGA-6/OMEGA-3 RATIO,OMEGA_3_TOTAL,OMEGA_6_TOTAL,PLATELET COUNT THOUSAND,POTASSIUM,"PROTEIN, TOTAL SERUM",RDW,RED CELL COUNT,SODIUM,TOTAL NEUTROPHILS,TOTAL NEUTROPHILS AB,TRIGLYCERIDES,Triglyceride HDL Ratio,UREA NITROGEN,URIC ACID,"VITAMIN D, 25-OH TOT",WHITE CELL COUNT
0,01074064,9.3,11.0,4.1,41.0,10.7,17.0,1.0,0.0,0.2,17.0,9.0,24.0,101.0,165.0,0.76,2.74,3.1,0.8,3.0,0.1,0.7,27.0,103.0,118.0,17.0,2.1,90.0,5.3,69.0,37.80,42.2,13.8,1.111111,11.4,0.0,0.0,5.0,965.0,328.0,81.0,20.9,24.2,25.0,44.0,1.8,1.9,30.7,32.7,94.0,1.6,163.0,7.0,0.3,4.6,8.5,4.6,38.9,229.0,4.5,6.2,13.0,4.49,138.0,45.0,1.8,75.0,1.086957,13.0,4.7,44.7,3.9
1,01920535,19.4,12.0,4.0,68.0,10.9,17.0,0.6,0.0,0.2,14.0,9.1,23.0,104.0,225.0,0.86,1.04,2.5,1.1,2.0,0.1,0.5,162.0,75.0,87.0,10.0,2.5,95.0,5.5,61.0,30.00,42.1,14.0,1.172840,11.4,0.0,0.0,5.0,1217.0,132.0,147.0,22.1,29.7,25.0,32.8,1.7,2.2,30.3,33.4,91.0,2.2,166.0,7.0,0.4,4.1,10.7,4.1,43.7,239.0,4.3,6.5,13.6,4.65,141.0,57.0,3.0,86.0,1.409836,12.0,5.5,17.8,5.3
2,01729568,17.6,57.0,4.3,49.0,11.6,46.0,0.0,0.0,0.7,15.0,9.5,25.0,106.0,234.0,0.85,1.22,3.3,1.4,3.0,0.1,1.0,196.0,68.0,78.0,17.0,1.7,91.0,5.3,62.0,33.80,40.7,13.9,2.134568,7.8,0.0,0.0,9.5,1771.0,419.0,153.0,21.7,24.4,25.0,44.0,1.9,2.1,32.2,34.2,94.0,1.7,180.0,10.0,0.4,5.7,7.0,5.7,39.8,219.0,4.3,6.0,13.9,4.32,144.0,43.0,1.9,94.0,1.516129,13.0,4.8,44.2,4.3
3,01977894,7.2,39.0,4.8,73.0,11.8,34.0,0.6,0.0,0.4,14.0,9.9,26.0,99.0,202.0,1.37,0.90,3.6,1.4,2.0,0.1,1.3,83.0,68.0,78.0,16.0,2.7,91.0,5.5,48.0,31.90,42.1,14.0,1.213333,8.3,0.0,0.0,5.4,1643.0,417.0,142.0,21.6,24.2,46.0,32.8,1.7,2.0,30.3,33.4,91.0,2.0,144.0,7.0,0.4,6.3,6.1,6.3,38.4,239.0,4.6,7.5,13.6,4.65,139.0,57.0,3.0,59.0,1.229167,19.0,5.4,41.7,5.3
4,01627898,7.2,21.0,4.6,51.0,11.8,20.0,0.6,0.0,2.0,14.0,9.3,24.0,104.0,194.0,0.92,2.84,3.3,1.5,2.0,0.1,1.4,122.0,98.0,113.0,14.0,2.3,85.0,5.1,59.0,34.35,42.1,14.0,0.755556,12.1,0.0,0.0,3.6,1230.0,349.0,111.0,21.2,25.6,32.0,32.8,1.7,2.0,30.3,33.4,91.0,2.0,144.0,7.0,0.4,6.2,6.6,6.2,40.8,239.0,4.0,6.9,13.6,4.65,143.0,57.0,3.0,91.0,1.534483,13.0,7.1,51.8,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4876,01034328,30.7,46.0,3.6,205.0,7.5,86.0,0.0,0.0,0.5,19.0,9.2,28.0,101.0,179.0,0.64,3.24,3.2,0.9,3.0,0.1,0.7,102.0,96.0,111.0,309.0,2.4,84.0,4.9,85.0,24.90,34.3,11.5,1.576296,11.6,0.0,0.0,7.6,650.0,90.0,80.0,22.1,21.3,44.0,13.0,0.5,1.8,37.8,33.5,113.0,8.8,118.0,10.0,0.3,4.8,6.6,4.8,31.5,150.0,4.2,6.0,14.0,3.04,141.0,74.0,2.5,72.0,0.847059,12.0,6.7,25.8,3.5
4877,QIRL,16.3,15.0,4.1,77.0,9.6,23.0,1.0,0.0,0.3,17.0,9.0,20.0,99.0,184.0,0.64,4.20,3.3,1.6,2.0,0.1,1.8,41.0,108.0,125.0,35.0,2.6,96.0,5.2,88.0,37.50,40.7,13.7,1.161481,8.4,0.0,0.0,4.9,684.0,90.0,83.0,21.2,25.2,25.0,25.0,1.2,2.0,28.7,33.7,85.0,0.0,137.0,7.0,0.4,6.7,5.7,6.7,38.3,179.0,4.4,6.7,15.2,4.77,137.0,65.0,3.1,65.0,0.738636,11.0,3.5,33.3,4.9
4878,HX842645,7.2,30.0,4.7,49.0,8.3,33.0,0.6,0.0,0.6,16.0,9.2,23.0,100.0,272.0,0.99,0.72,2.1,1.1,2.0,0.1,0.7,110.0,96.0,110.0,36.0,2.6,90.0,5.2,34.0,24.30,42.1

In [8]:
#Get the gut microbiome shotgun metagenomic data:
#asvs = pd.read_csv("/proj/arivale/microbiome/16S_processed/asvs.csv")
asvs = pd.read_csv('asvs.csv')
#samples = pd.read_csv("/proj/arivale/microbiome/16S_processed/metadata.csv")
samples = pd.read('metadata.csv')
merged = pd.merge(asvs, samples[["id", "public_client_id", "sex", "age", "days_in_program"]], on="id")
#tax = pd.read_csv("/proj/arivale/microbiome/16S_processed/taxonomy.csv")
tax = pd.read_csv('taxonomy.csv')
tax = tax.drop("sequence", axis=1).rename(columns={"id": "hash"})
merged = pd.merge(merged, tax, on="hash")

#Create taxa genus df
taxa = merged[['public_client_id','count','Kingdom','Phylum','Class','Order','Family','Genus'#,'Species'
              ]]

#Add prefixes to taxa names for consistency
taxa = taxa.assign(Name = 'taxa_' + taxa.Kingdom.astype(str) + '.' + \
  taxa.Phylum.astype(str) + '.' + taxa.Class.astype(str) + '.' + \
  taxa.Order.astype(str) + '.' + taxa.Family.astype(str) + '.' + \
  taxa.Genus.astype(str)# + '.' + taxa.Species.astype(str) \
                  )

taxa = taxa.sort_values(by = ['public_client_id'])

#create MultiIndex for pivot table
arrays = [taxa['public_client_id'],taxa['Name']]
index = pd.MultiIndex.from_arrays(arrays, names = ('public_client_id','Name'))

#pivot df into one with taxa as columns and PCI as index with counts as values
name_melt = pd.DataFrame(data = {'count':taxa['count'].to_numpy()}, index = index)

name_pivot = name_melt.pivot_table(index='public_client_id', columns='Name', values='count',aggfunc='sum', fill_value = 0)
name_pivot = name_pivot.dropna(how = 'any')

# compute CLR-transformed taxa matrix
clr_pivot = name_pivot + 1 #add psuedocount to logarithmically compute downstream
from skbio.stats.composition import clr
clr_pivot = pd.DataFrame(clr(clr_pivot), index =clr_pivot.index, columns = clr_pivot.columns)
clr_pivot.to_csv('clrtaxa.csv')

taxa = name_pivot
taxa_zerocounts=(taxa == 0).sum() # sum up times any columns have zero values
taxa_nonnancounts=(~taxa.isna()).sum() # sum up times any columns have non-NA values
taxa = taxa.loc[:, taxa_zerocounts/taxa.shape[0] <= 0.70] #this sum of non-NA count values/num of total rows should be greater than 70% to proceed with the cohort
clr_taxa = taxa + 1 # readding the pseudocount
clr_taxa = pd.DataFrame(clr(clr_taxa), index = clr_taxa.index, columns = clr_taxa.columns)
taxa = taxa.reset_index()
clr_taxa = clr_taxa.reset_index()
taxa.public_client_id = taxa.public_client_id.values.astype(str)
clr_taxa.public_client_id = clr_taxa.public_client_id.values.astype(str)

# Obtain vendor from snapshot
dash = adi.get_snapshot('microbiome_diversity')
dash = dash.sort_values('days_in_program',ascending=True) # Sort by days in program first
#dash = dash.drop_duplicates(subset='public_client_id') # drop duplicate PCIs, keeping first day
dash = dash.sort_values('public_client_id',ascending=True)
dash = dash.set_index('public_client_id')
dash = dash[['vendor_dashboard']]
dash = dash[dash.notna()]
dash = dash.reset_index()
taxa
#3694 participants

Name,public_client_id,taxa_Archaea.Euryarchaeota.Methanobacteria.Methanobacteriales.Methanobacteriaceae.Methanobrevibacter,taxa_Bacteria.Actinobacteria.Actinobacteria.Actinomycetales.Actinomycetaceae.Actinomyces,taxa_Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Bifidobacterium,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Coriobacteriaceae.Collinsella,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Coriobacteriales_Incertae_Sedis.nan,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Eggerthellaceae.Adlercreutzia,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Eggerthellaceae.DNF00809,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Eggerthellaceae.Eggerthella,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Eggerthellaceae.Gordonibacter,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Eggerthellaceae.nan,taxa_Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.nan.nan,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Bacteroidaceae.Bacteroides,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Barnesiellaceae.Barnesiella,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Barnesiellaceae.Coprobacter,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Marinifilaceae.Butyricimonas,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Marinifilaceae.Odoribacter,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Muribaculaceae.nan,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Prevotellaceae.Prevotella_9,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Rikenellaceae.Alistipes,taxa_Bacteria.Bacteroidetes.Bacteroidia.Bacteroidales.Tannerellaceae.Parabacteroides,taxa_Bacteria.Cyanobacteria.Melainabacteria.Gastranaerophilales.nan.nan,taxa_Bacteria.Cyanobacteria.Oxyphotobacteria.Chloroplast.nan.nan,taxa_Bacteria.Firmicutes.Bacilli.Bacillales.Family_XI.Gemella,taxa_Bacteria.Firmicutes.Bacilli.Lactobacillales.Carnobacteriaceae.Granulicatella,taxa_Bacteria.Firmicutes.Bacilli.Lactobacillales.Lactobacillaceae.Lactobacillus,taxa_Bacteria.Firmicutes.Bacilli.Lactobacillales.Streptococcaceae.Lactococcus,taxa_Bacteria.Firmicutes.Bacilli.Lactobacillales.Streptococcaceae.Streptococcus,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Christensenellaceae.Christensenellaceae_R-7_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Christensenellaceae.nan,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Clostridiaceae_1.Clostridium_sensu_stricto_1,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Clostridiales_vadinBB60_group.nan,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Defluviitaleaceae.Defluviitaleaceae_UCG-011,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Eubacteriaceae.Anaerofustis,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Family_XIII.Family_XIII_AD3011_group,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Family_XIII.Family_XIII_UCG-001,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Family_XIII.nan,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Agathobacter,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Anaerostipes,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Blautia,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.CAG-56,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Coprococcus_1,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Coprococcus_2,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Coprococcus_3,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Dorea,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Eisenbergiella,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Fusicatenibacter,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.GCA-900066575,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.Lachnospiraceae.Hungatella,taxa_Bacteria.Firmicutes.Clostridia.Clostridiales.La

In [43]:
# Taxonomy metadata:
# Algorithm provided by Christian Diener, PhD:
###################################################
taxa_table = taxa.iloc[:,1:]
taxa_table = taxa_table.columns  # to get only the taxa columns
taxa_table = taxa_table[taxa_table.str.startswith("taxa_")]
taxa_table = taxa_table.str.replace("taxa_", "")  # to remove the leading `genus_`
taxa_table = taxa_table.to_series().str.split(".", expand=True)# will split on the dot and save it into the columns of a new DataFrame
taxa_table.columns = ["kingdom", "phylum", "class", "order", "family", "genus"
                     ]# set the column names

taxa_table.to_csv("taxa.csv", sep="\t")
####################################################
taxa_table

,kingdom,phylum,class,order,family,genus
Name,,,,,,
Archaea.Euryarchaeota.Methanobacteria.Methanobacteriales.Methanobacteriaceae.Methanobrevibacter,Archaea,Euryarchaeota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter
Bacteria.Actinobacteria.Actinobacteria.Actinomycetales.Actinomycetaceae.Actinomyces,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Actinomyces
Bacteria.Actinobacteria.Actinobacteria.Bifidobacteriales.Bifidobacteriaceae.Bifidobacterium,Bacteria,Actinobacteria,Actinobacteria,Bifidobacteriales,Bifidobacteriaceae,Bifidobacterium
Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Coriobacteriaceae.Collinsella,Bacteria,Actinobacteria,Coriobacteriia,Coriobacteriales,Coriobacteriaceae,Collinsella
Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Coriobacteriales_Incertae_Sedis.nan,Bacteria,Actinobacteria,Coriobacteriia,Coriobacteriales,Coriobacteriales_Incertae_Sedis,nan
Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Eggerthellaceae.Adlercreutzia,Bacteria,Actinobacteria,Coriobacteriia,Coriobacteriales,Eggerthellaceae,Adlercreutzia
Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Eggerthellaceae.DNF00809,Bacteria,Actinobacteria,Coriobacteriia,Coriobacteriales,Eggerthellaceae,DNF00809
Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Eggerthellaceae.Eggerthella,Bacteria,Actinobacteria,Coriobacteriia,Coriobacteriales,Eggerthellaceae,Eggerthella
Bacteria.Actinobacteria.Coriobacteriia.Coriobacteriales.Eggerthellaceae.Gordonibacter,Bacteria,Actinobacteria,Coriobacteriia,Coriobacteriales,Eggerthellaceae,Gordonibacter


In [42]:
#Sanity Check
#Check for bacteroidetes before and after filtering, since it is so abundant and important.
words = name_pivot.columns
words = words.to_series().str.split(".",expand=True)
family_name = words.iloc[:,4]
genus_name = words.iloc[:,5]
family_name_values = family_name.values
genus_name_values = genus_name.values
i = 0
for col in genus_name_values:
    i += 1
    if 'Bacteroides' == col:
       print("Yes, Bacteroides are in the samples")
       print("Index number: ",i)
       print("Family: ", family_name_values[i])
       print("Genus: ", genus_name_values[i])

Yes, Bacteroides are in the samples
Index number:  91
Family:  Bacteroidales_Incertae_Sedis
Genus:  Phocaeicola


In [17]:
#get the proteomics dataframe:
pro = adi.get_snapshot('proteomics_corrected')
pro = pro.sort_values(by='days_in_program')

pro = pro.drop_duplicates('public_client_id')
pro = pro.set_index('public_client_id')
pro = pro.iloc[:,14:]
pro = pro.drop(columns=['month','weekday','season'])
pro_zerocounts=(pro == 0).sum() # sum up times any columns have zero values
pro_nonnancounts=(~pro.isna()).sum() # sum up times any columns have non-NA values
pro = pro.loc[:, pro_nonnancounts/pro.shape[0] >= .50] #this sum of non-NA count values/num of total rows should be >= 50% to proceed with the cohort
pro = pro.fillna(pro.median()) #impute with the median
pro = pro.reset_index('public_client_id')

#Get the proteomics metadata df:
pro_m = adi.get_snapshot('proteomics_metadata')
pro_m = pro_m.drop_duplicates()
common_rows = set(pro.columns)
common_columns = set(pro_m['name'])
common = pd.Series(list(common_rows.intersection(common_columns)))
pro_m = pro_m[pro_m['name'].isin(common)]
pro_m.to_csv('proteomics_metadata_table.csv', index = False)

pro
#2859 participants

,public_client_id,days_in_program,days_since_first_call,days_since_first_draw,CVD2_O00182,CVD2_O00220,CVD2_O00253,CVD2_O14763,CVD2_O14836,CVD2_O43915,CVD2_O94907,CVD2_P00797,CVD2_P01127,CVD2_P01241,CVD2_P01730,CVD2_P01833,CVD2_P02760,CVD2_P04179,CVD2_P04792,CVD2_P05231,CVD2_P06858,CVD2_P07204,CVD2_P07585,CVD2_P07711,CVD2_P09237,CVD2_P09341,CVD2_P09601,CVD2_P09874,CVD2_P10147,CVD2_P12104,CVD2_P12931,CVD2_P13726,CVD2_P16860,CVD2_P18510,CVD2_P19883,CVD2_P21583,CVD2_P21980,CVD2_P22004,CVD2_P24394,CVD2_P25116,CVD2_P26022,CVD2_P27352,CVD2_P29965,CVD2_P31994,CVD2_P31997,CVD2_P35218,CVD2_P35318,CVD2_P35442,CVD2_P35475,CVD2_P39900,CVD2_P40225,CVD2_P41159,CVD2_P47992,CVD2_P49763,CVD2_P51161,CVD2_P51888,CVD2_P78380,CVD2_Q02763,CVD2_Q04760,CVD2_Q12866,CVD2_Q13043,CVD2_Q13219,CVD2_Q14005,CVD2_Q14116,CVD2_Q14242,CVD2_Q15109,CVD2_Q15389,CVD2_Q16651,CVD2_Q16698,CVD2_Q76LX8,CVD2_Q8IW75,CVD2_Q8IYS5,"CVD2_Q8NEV9,Q14213",CVD2_Q8TAD2,CVD2_Q92583,CVD2_Q96D42,CVD2_Q96IQ7,CVD2_Q99075,CVD2_Q99523,CVD2_Q99895,CVD2_Q9BQ51,CVD2_Q9BQR3,CVD2_Q9BUD6,CVD2_Q9BWV1,CVD2_Q9BYF1,CVD2_Q9GZV9,CVD2_Q9HB29,CVD2_Q9NQ25,CVD2_Q9NSA1,CVD2_Q9UEW3,CVD2_Q9UIB8,CVD2_Q9UJM8,CVD2_Q9UK05,CVD2_Q9UKP3,CVD2_Q9Y6K9,CVD2_Q9Y6Q6,CVD3_NT-proBNP,CVD3_O00175,CVD3_O00300,CVD3_O14798,...,CVD3_Q9H2A7,CVD3_Q9HCB6,CVD3_Q9HD89,CVD3_Q9NPY3,CVD3_Q9NQ76,CVD3_Q9UBR2,CVD3_Q9UM47,CVD3_Q9Y275,CVD3_Q9Y624,INF_O00300,INF_O14625,INF_O14788,INF_O15169,INF_O15444,INF_O43508,INF_O43557,INF_O95630,INF_O95750,INF_O95760,INF_P00749,INF_P00813,INF_P01135,INF_P01137,INF_P01138,INF_P01374,INF_P01375,INF_P01579,INF_P01583,INF_P02778,INF_P03956,INF_P05112,INF_P05113,INF_P05231,INF_P06127,INF_P09238,INF_P09341,INF_P09603,INF_P10145,INF_P10147,INF_P13232,INF_P13236,INF_P13500,INF_P13725,INF_P14210,INF_P14784,INF_P15018,INF_P15692,INF_P20783,INF_P21583,INF_P22301,INF_P25942,INF_P28325,INF_P29460,INF_P35225,INF_P39905,INF_P42702,INF_P42830,INF_P49771,INF_P50225,INF_P50591,INF_P51671,INF_P55773,INF_P60568,INF_P78423,INF_P78556,INF_P80075,INF_P80098,INF_P80162,INF_P80511,INF_Q07011,INF_Q07325,INF_Q08334,INF_Q13007,INF_Q13261,INF_Q13291,INF_Q13478,INF_Q13541,INF_Q13651,INF_Q14116,INF_Q14790,INF_Q16552,INF_Q5T4W7,INF_Q8IXJ6,INF_Q8N6P7,INF_Q8NF90,INF_Q8NFT8,INF_Q8WWJ7,INF_Q969D9,INF_Q99616,INF_Q99731,INF_Q99748,INF_Q9BZW8,INF_Q9GZV9,INF_Q9H5V8,INF_Q9NRJ3,INF_Q9NSA1,INF_Q9NYY1,INF_Q9NZQ7,INF_Q9P0M4,INF_Q9UHF4
0,01324699,0,0.0,0,9.231985,2.592163,4.438840,5.415310,9.117643,7.589893,9.562513,8.300667,11.069715,6.473435,5.561110,7.255285,8.283945,9.852695,11.452455,3.211300,10.077635,10.447515,5.070775,7.729768,8.528125,9.854055,12.027368,3.165315,2.339105,9.423150,8.109230,7.062770,1.367710,3.249165,11.596135,10.620235,8.838168,5.500822,2.392595,9.634272,3.766472,6.748795,7.775143,3.388507,3.500265,2.314660,7.373947,6.007297,6.213300,6.991492,3.824630,3.732428,5.361755,8.255590,2.468195,6.961272,6.388730,8.067500,6.832533,6.330823,7.276130,2.972825,6.665920,8.616800,6.059638,5.386335,9.770857,8.925230,7.721185,5.933065,4.796880,10.739845,5.268065,2.728345,8.884775,7.164013,3.020065,5.423430,8.242772,10.359713,3.078232,9.943740,10.864230,5.554510,3.449190,2.354325,4.355335,3.116550,3.264810,7.446648,6.594798,1.785605,7.370555,5.521570,7.470175,5.868290,3.974060,6.916073,4.028932,6.941217,...,5.947933,1.381960,6.852100,11.709380,5.949363,5.354328,4.825025,7.133913,7.167890,10.112685,7.499920,3.927450,5.415790,5.295350,11.034075,4.161560,5.672920,8.066210,1.466040,9.880740,4.357900,3.321510,6.845095,2.353550,4.729975,0.363915,0.834370,1.702635,8.079620,13.062245,0.065260,-0.699615,2.432660,4.559845,6.090595,10.722110,9.919035,5.474930,1.191375,3.030650,2.412410,10.767175,3.800345,8.628870,1.559440,-0.587385,9.735700,2.694700,10.120920,3.525400,10.940965,6.653205,4.938755,-0.134755,2.296010,3.450235,11.737640,10.008455,4.064375,8.529485,8.174590,9.877175,0.244235,6.417265,0.832780,8.409350,2.417270,7.924395,1.883505,6.830055,6.769730,6.446970,-0.145285,0.696165,3.101615,7.466350,7.782365,-0.137720,8.643360,3.167800,0.762185,0.09

In [18]:
# get the covariates for CRP levels, LDL cholesterol, and hbA1C:
blood = pd.merge(dip,crp,on=['public_client_id','days_in_program'])
blood = pd.merge(blood,ldl,on=['public_client_id','days_in_program'])
blood = pd.merge(blood,a1c,on=['public_client_id','days_in_program'])
blood = blood.drop('days_in_program', axis=1)
blood
#4881 participants

,public_client_id,CRP HIGH SENSITIVITY,LDL-CHOL CALCULATION,GLYCOHEMOGLOBIN A1C
0,01074064,2.74,81.0,5.3
1,01920535,1.04,147.0,5.5
2,01729568,1.22,153.0,5.3
3,01977894,0.90,142.0,5.5
4,01627898,2.84,NaN,5.1
...,...,...,...,...
4876,01034328,3.24,80.0,4.9
4877,QIRL,4.20,83.0,5.2
4878,HX842645,0.72,170.0,5.2
4879,01232614,0.98,115.0,5.4


In [19]:
#Get the clinical laboratory chemistries data from the snapshot:
chem=adi.get_snapshot('chemistries')
chemdf = chem.sort_values(by='days_in_program') #keep only first date data
#chemdf = chemdf.drop_duplicates(subset='public_client_id')
chemdf = chemdf.set_index('public_client_id')
chemdf = chemdf.iloc[:,12:] #keep only relevant columns
chem_nonnancounts=(~chemdf.isna()).sum() # sum up times any columns have non-NA values
chemdf = chemdf.loc[:, chem_nonnancounts/chemdf.shape[0] >= .70] #this sum of non-NA values/num of total rows should be >= 70% to proceed with the cohort
chemdf = chemdf.fillna(chemdf.median())
chemdf = chemdf.dropna()
chemdf = chemdf.reset_index()
chemdf 
#11167 participants

,public_client_id,ALAT (SGPT),ALBUMIN,ALKALINE PHOSPHATASE,ARACHIDONIC ACID,ASAT (SGOT),BASOPHILS,BASOPHILS ABSOLUTE,"BILIRUBIN, TOTAL",BUN/CREAT RATIO,CALCIUM,CARBON DIOXIDE (CO2),CHLORIDE,"CHOLESTEROL, TOTAL","CREATININE ENZ, SER",CRP HIGH SENSITIVITY,DHA,DPA,EOSINOPHILS,EOSINOPHILS ABSOLUTE,EPA,FERRITIN,"GFR, MDRD","GFR, MDRD, AFRICAN AM",GGT,GLOBULIN,GLUCOSE,GLYCOHEMOGLOBIN A1C,HDL CHOL DIRECT,HDL PARTICLE NUMBER,HEMATOCRIT,HEMOGLOBIN,HOMA-IR,"HOMOCYSTEINE, SERUM",IMMATURE GRANULOCYTES,IMMATURE GRANULOCYTES ABSOLUTE,INSULIN,LDL PARTICLE NUMBER,LDL SMALL,LDL-CHOL CALCULATION,LDL_SIZE,LINOLEIC_ACID,LPIR_SCORE,LYMPHOCYTES,LYMPHOCYTES ABSOLUTE,MCH,MCHC,MCV,"MERCURY, BLOOD",MONOCYTES,MONOCYTES ABSOLUTE,OMEGA-3 INDEX,OMEGA-6/OMEGA-3 RATIO,OMEGA_3_TOTAL,OMEGA_6_TOTAL,PLATELET COUNT THOUSAND,POTASSIUM,"PROTEIN, TOTAL SERUM",RDW,RED CELL COUNT,SODIUM,TOTAL NEUTROPHILS,TOTAL NEUTROPHILS AB,TRIGLYCERIDES,Triglyceride HDL Ratio,UREA NITROGEN,URIC ACID,"VITAMIN D, 25-OH TOT",WHITE CELL COUNT
0,01074064,11.0,4.1,41.0,10.7,17.0,1.0,0.0,0.2,17.0,9.0,24.0,101.0,165.0,0.76,2.74,3.1,0.8,3.0,0.1,0.7,27.0,103.0,118.0,16.0,2.1,90.0,5.3,69.0,37.8,42.2,13.8,1.111111,11.4,0.0,0.0,5.0,965.0,328.0,81.0,20.9,24.2,25.0,44.0,1.8,30.7,32.7,94.0,1.6,7.0,0.3,4.6,8.5,4.6,38.9,229.0,4.5,6.2,13.0,4.49,138.0,45.0,1.8,75.0,1.086957,13.0,4.7,44.7,3.9
1,01074064,17.0,4.3,47.0,10.7,26.0,1.0,0.0,0.4,16.0,8.8,23.0,102.0,193.0,0.77,1.36,2.7,0.8,3.0,0.1,0.6,20.0,101.0,117.0,16.0,2.4,83.0,5.3,76.0,43.5,40.1,13.6,0.696790,12.1,0.0,0.0,3.4,973.0,309.0,101.0,21.0,24.1,32.0,47.0,1.8,31.2,33.9,92.0,1.6,5.0,0.2,4.1,9.4,4.1,38.4,241.0,4.0,6.7,12.9,4.36,141.0,44.0,1.8,78.0,1.026316,12.0,5.4,41.3,4.0
2,01920535,12.0,4.0,68.0,10.9,17.0,1.0,0.0,0.2,14.0,9.1,23.0,104.0,225.0,0.86,1.04,2.5,1.1,2.0,0.1,0.5,162.0,75.0,87.0,10.0,2.5,95.0,5.5,61.0,30.0,41.9,14.0,1.172840,11.4,0.0,0.0,5.0,1217.0,132.0,147.0,22.1,29.7,25.0,33.0,1.7,30.4,33.5,91.0,2.2,7.0,0.4,4.1,10.7,4.1,43.7,236.0,4.3,6.5,13.6,4.63,141.0,56.0,2.9,86.0,1.409836,12.0,5.5,17.8,5.2
3,01729568,57.0,4.3,49.0,11.6,46.0,0.0,0.0,0.7,15.0,9.5,25.0,106.0,234.0,0.85,1.22,3.3,1.4,3.0,0.1,1.0,196.0,68.0,78.0,17.0,1.7,91.0,5.3,62.0,33.8,40.7,13.9,2.134568,7.8,0.0,0.0,9.5,1771.0,419.0,153.0,21.7,24.4,25.0,44.0,1.9,32.2,34.2,94.0,1.7,10.0,0.4,5.7,7.0,5.7,39.8,219.0,4.3,6.0,13.9,4.32,144.0,43.0,1.9,94.0,1.516129,13.0,4.8,44.2,4.3
4,01977894,39.0,4.8,73.0,11.8,34.0,1.0,0.0,0.4,14.0,9.9,26.0,99.0,202.0,1.37,0.90,3.6,1.4,2.0,0.1,1.3,83.0,68.0,78.0,16.0,2.7,91.0,5.5,48.0,31.9,41.9,14.0,1.213333,8.3,0.0,0.0,5.4,1643.0,417.0,142.0,21.6,24.2,46.0,33.0,1.7,30.4,33.5,91.0,2.1,7.0,0.4,6.3,6.1,6.3,38.4,236.0,4.6,7.5,13.6,4.63,139.0,56.0,2.9,59.0,1.229167,19.0,5.4,41.7,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,01194014,27.0,4.4,52.0,10.9,23.0,1.0,0.1,1.2,18.0,10.0,23.0,104.0,172.0,0.72,0.63,4.4,2.4,7.0,0.5,5.4,88.0,104.0,120.0,19.0,2.5,99.0,5.2,92.0,37.9,39.1,12.8,2.224444,9.4,0.0,0.0,9.1,652.0,90.0,70.0,21.0,22.1,25.0,40.0,2.5,30.2,32.7,92.0,2.1,7.0,0.4,12.2,2.9,12.2,35.2,321.0,4.3,6.9,13.8,4.24,141.0,45.0,2.8,51.0,0.554348,13.0,3.8,44.5,6.2
11163,01373090,36.0,4.4,64.0,10.2,24.0,0.0,0.0,0.9,23.0,9.7,24.0,98.0,250.0,0.79,1.30,4.2,0.9,3.0,0.1,1.5,96.0,84.0,97.0,29.0,2.5,90.0,5.6,105.0,38.6,36.7,12.3,0.866667,6.3,0.0,0.0,3.9,1264.0,90.0,132.0,21.6,26.1,25.0,32.0,1.7,29.0,33.5,87.0,0.0,7.0,0.4,6.6,5.8,6.6,38.6,282.0,4.3,6.9,14.2,4.24,137.0,58.0,3.1,66.0,0.628571,18.0,4.7,52.1,5.3
11164,01901634,27.0,4.4,48.0,12.2,26.0,0.0,0.0,0.7,15.0,9.6,27.0,99.0,188.0,1.14,0.42,3.8,1.8,3.0,0.1,2.7,88.0,78.0,91.0,11.0,2.3,95.0,5.4,63.0,33.3,44.2,14.6,1.243210,9.4,0.0,0.0,5.3,1330.0,389.0,114.0,21.0,25.0,30.0,40.0,1.5,30.0,33.0,91.0,8.7,8.0,0.3,8.3,4.8,8.3,39.5,158.0,4.6,6.7,14.3,4.86,139.0,49.0,1.8,54.0,0.857143,17.0,4.9,

In [20]:
# Get PSS stress data
# code adapted from Annie Levine, MD:
####################################################################################################
assessments = adi.get_snapshot('assessments', clean=True)
stress = adi.get_snapshot("assessments")
stress = stress.sort_values('days_in_program',ascending=True)
stress = stress.drop(columns = ['vendor','days_in_program','days_since_first_call','days_since_first_draw'])
stress = stress.drop_duplicates(subset='public_client_id')
stress = stress.fillna(0.001)
stress = stress.set_index('public_client_id')
stress = stress[["assessment:pss-four-item:out_of_control:enum", "assessment:pss-four-item:handle_problems:enum", "assessment:pss-four-item:going_your_way:enum", "assessment:pss-four-item:insurmountable:enum"]] 
stress = stress.rename(columns={"assessment:pss-four-item:out_of_control:enum" : "pss_1", "assessment:pss-four-item:handle_problems:enum": "pss_2", "assessment:pss-four-item:going_your_way:enum": "pss_3", "assessment:pss-four-item:insurmountable:enum": "pss_4"})

# Coding question values and replacing with numeric values
pss_1_list = ['(0) Never', '(1) Almost Never', '(2) Sometimes', '(3) Fairly Often', '(4) Very Often']
pss_1_r = [0, 1, 2, 3, 4]
stress["pss_1"] = stress["pss_1"].replace(pss_1_list, pss_1_r)
pss_2_list = ['(0) Very Often', '(1) Fairly Often', '(2) Sometimes', '(3) Almost Never', '(4) Never']
pss_2_r = [0, 1, 2, 3, 4] 
stress["pss_2"] = stress["pss_2"].replace(pss_2_list, pss_2_r)
pss_3_list = ['(0) Very Often', '(1) Fairly Often', '(2) Sometimes', '(3) Almost Never', '(4) Never']
pss_3_r = [0, 1, 2, 3, 4] 
stress["pss_3"] = stress["pss_3"].replace(pss_3_list, pss_3_r)
pss_4_list = ['(0) Never', '(1) Almost Never', '(2) Sometimes', '(3) Fairly Often', '(4) Very Often']
pss_4_r = [0, 1, 2, 3, 4]
stress["pss_4"] = stress["pss_4"].replace(pss_4_list, pss_4_r)

#Calculating the total stress score
def stress_add (row):
    stress_score = row["pss_1"] + row["pss_2"] + row["pss_3"] + row["pss_4"]
    return stress_score
stress.apply (lambda row: stress_add(row), axis=1)
stress["stress_score"] = stress.apply (lambda row: stress_add(row), axis=1)
stress

#Make it a DataFrame and remove NaNs
stress = stress[stress.stress_score !=0.004]
stress = stress.reset_index()
####################################################################################################

stress
#4725 participants

,public_client_id,pss_1,pss_2,pss_3,pss_4,stress_score
0,01071494,2.0,0.0,1.0,1.0,4.0
1,01092042,0.0,0.0,1.0,0.0,1.0
2,01568974,1.0,0.0,1.0,1.0,3.0
3,01684342,2.0,1.0,1.0,2.0,6.0
4,01601230,1.0,1.0,3.0,1.0,6.0
...,...,...,...,...,...,...
4720,01232614,2.0,1.0,1.0,1.0,5.0
4721,01181682,0.0,0.0,0.0,0.0,0.0
4722,01142874,0.0,4.0,0.0,0.0,4.0
4723,01380110,3.0,1.0,2.0,2.0,8.0


In [21]:
#Now, get the survey data/"assessments" data:
bio=adi.get_snapshot('clients')
a = bio['sex']
b = bio['age']
c = bio['public_client_id']
biodf = pd.DataFrame({'sex':a, 'age':b, 'public_client_id':c})
biodf = biodf.drop_duplicates(subset='public_client_id')
biodf = biodf[['sex','age','public_client_id']]
biodf

#Get BMI data
bmi = adi.get_snapshot('weight_monthly')
bmi = bmi[['public_client_id','BMI_CALC']]
bmi = bmi.sort_values('public_client_id',ascending=True)
bmi = bmi.drop_duplicates('public_client_id')
bmi = bmi.dropna()
bmi #5580 individuals

,public_client_id,BMI_CALC
0,01000261,22.594123
3,01000552,31.619378
5,01001181,39.047808
6,01001298,26.622485
12,01001467,18.479394
...,...,...
46532,QIRL,37.286232
46563,ZU273983,30.984144
46568,ZU612255,35.177122
46569,ZU621944,51.577264


In [22]:
# Begin defining exclusion criteria
# These criteria will exclude participants from the cohort that meet certain conditions in order to focus on a "generally healthy" cohort
laxmed = adi.get_snapshot('assessments') # get info on those taking laxatives
laxmed = laxmed[['public_client_id','days_in_program','assessment:digestion:laxatives:enum']]
cholmed = adi.get_snapshot('assessments_medications') # get info on those taking cholesterol/BP meds
cholmed = cholmed[['public_client_id','days_in_program','meds_cholesterol','meds_blood_pressure']]
laxmed = laxmed.rename(columns={'assessment:digestion:laxatives:enum':'meds_laxatives'})
meds = pd.merge(laxmed,cholmed)
meds # keep this df for exclusion criteria

# assemble another exclusion criteria df:
hh = adi.get_snapshot('assessments_health_history_new')
dig = adi.get_snapshot('assessments_digestive_health')

#Retrieve miscellaneous digestive health indicators: 
lifestyle = adi.get_snapshot('assessments') 
lifestyle = lifestyle.drop_duplicates(subset='public_client_id')
lifestyle = lifestyle.sort_values('days_in_program',ascending=True)
meds = meds[['public_client_id','meds_laxatives','meds_cholesterol','meds_blood_pressure']]

digest = pd.merge(hh,dig, on = list(['vendor','days_in_program','days_since_first_call','days_since_first_draw','month','weekday','season',"public_client_id"]))
digest = pd.merge(digest, meds, on = "public_client_id")

common_columns = list(set(digest.columns).intersection(lifestyle.columns))
digest = pd.merge(digest, lifestyle, on=common_columns) # exclusion criteria df

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.11/site-packages/arivale_data_interface/snapshots_interface.py:271: DeprecationWarning: `assessments_medications` is slated for deprecation and will be part of `assessments` with raw data will be available in `assessments_raw`
  check_deprecation(name) # check if requested snapshot is to be deprecated
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.11/site-packages/arivale_data_interface/snapshots_interface.py:271: DeprecationWarning: `assessments_health_history_new` is slated for deprecation and will be part of `assessments` with raw data will be available in `assessments_raw`
  check_deprecation(name) # check if requested snapshot is to be deprecated
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.11/site-packages/arivale_data_interface/snapshots_interface.py:271: DeprecationWarning: `assessments_digestive_health` is slated for deprecation and will be part of `assessments` with raw data will be available in `assessm

In [23]:
# create a list of values to check
values = ['Yes', 'Y', 'True','TRUE',True, 1
          ,'(3) Once per week or less', '(1) Daily','(2) Several times per week', '(2) Currently'
         ]

substrings_to_detect = ['public_client_id','meds_blood_pressure','drugs_cholesterol', 'meds_cholesterol','meds_laxatives', 'ulcerative_colitis', 'kidney_disease','bladder_kidney','kidney_cancer','kidney_failure','celiac_disease','colonic_crohns','ileal_crohns','diverticulosis','inflammatory_bowel','irritable_bowel','gerd','peptic_ulcer','coagulation']
substrings_to_exclude = ['siblings','sister','mother','family','father','paternal','maternal','grandparent','self_past','gt1ya']
inclusion_mask = digest.columns.str.contains('|'.join(substrings_to_detect))
exclusion_mask = digest.columns.str.contains('|'.join(substrings_to_exclude))

self_and_no_family_exclusion = inclusion_mask & ~exclusion_mask
ignored_features = ~(np.insert(inclusion_mask[1:],0,False) & ~exclusion_mask)
digest_excluded_w_self_and_no_family = digest.loc[:, self_and_no_family_exclusion]  # Selects columns based on the mask

matching_indexes = digest_excluded_w_self_and_no_family[digest_excluded_w_self_and_no_family.isin(values).any(axis=1)].public_client_id.values
digest_excluded_w_self_and_no_family_excluded = digest.loc[digest.public_client_id.isin(matching_indexes)].drop_duplicates(subset='public_client_id')

digest_excluded_w_self_and_no_family_excluded.loc[:, ignored_features]
result = digest_excluded_w_self_and_no_family_excluded['public_client_id']
print(len(bowel[bowel['public_client_id'].isin(result)])) #those with self history of kidney disease, med exclusions, and GI disease

988


In [24]:
# create a list of values to check
values = ['Yes', 'Y', 'True','TRUE',True, 1
          ,'(3) Once per week or less', '(1) Daily','(2) Several times per week', '(2) Currently'
         ]

substrings_to_detect = ['public_client_id','kidney_disease','bladder_kidney','kidney_cancer','kidney_failure']
substrings_to_exclude = ['siblings','sister','mother','family','father','paternal','maternal','grandparent','self_past','gt1ya']
inclusion_mask = digest.columns.str.contains('|'.join(substrings_to_detect))
exclusion_mask = digest.columns.str.contains('|'.join(substrings_to_exclude))

self_and_no_family_exclusion = inclusion_mask & ~exclusion_mask
ignored_features = ~(np.insert(inclusion_mask[1:],0,False) & ~exclusion_mask)
digest_excluded_w_self_and_no_family = digest.loc[:, self_and_no_family_exclusion]  # Selects columns based on the mask

matching_indexes = digest_excluded_w_self_and_no_family[digest_excluded_w_self_and_no_family.isin(values).any(axis=1)].public_client_id.values
digest_excluded_w_self_and_no_family_excluded = digest.loc[digest.public_client_id.isin(matching_indexes)].drop_duplicates(subset='public_client_id')

digest_excluded_w_self_and_no_family_excluded.loc[:, ignored_features]
result = digest_excluded_w_self_and_no_family_excluded['public_client_id']
#print(result)
print(len(bowel[bowel['public_client_id'].isin(result)])) #those with self history of kidney disease

119


In [25]:
# create a list of values to check
values = ['Yes', 'Y', 'True','TRUE',True, 1
          ,'(3) Once per week or less', '(1) Daily','(2) Several times per week', '(2) Currently'
         ]

substrings_to_detect = ['public_client_id','inflammatory_bowel','irritable_bowel']
substrings_to_exclude = ['siblings','sister','mother','family','father','paternal','maternal','grandparent','self_past','gt1ya']
inclusion_mask = digest.columns.str.contains('|'.join(substrings_to_detect))
exclusion_mask = digest.columns.str.contains('|'.join(substrings_to_exclude))

self_and_no_family_exclusion = inclusion_mask & ~exclusion_mask
ignored_features = ~(np.insert(inclusion_mask[1:],0,False) & ~exclusion_mask)
digest_excluded_w_self_and_no_family = digest.loc[:, self_and_no_family_exclusion]  # Selects columns based on the mask

matching_indexes = digest_excluded_w_self_and_no_family[digest_excluded_w_self_and_no_family.isin(values).any(axis=1)].public_client_id.values
digest_excluded_w_self_and_no_family_excluded = digest.loc[digest.public_client_id.isin(matching_indexes)].drop_duplicates(subset='public_client_id')

digest_excluded_w_self_and_no_family_excluded.loc[:, ignored_features]
result = digest_excluded_w_self_and_no_family_excluded['public_client_id']
#print(result)
print(len(bowel[bowel['public_client_id'].isin(result)])) #those with self history of IBD/IBS

128


In [26]:
# create a list of values to check
values = ['Yes', 'Y', 'True','TRUE',True, 1
          ,'(3) Once per week or less', '(1) Daily','(2) Several times per week', '(2) Currently'
         ]

substrings_to_detect = ['public_client_id','meds_blood_pressure','drugs_cholesterol', 'meds_cholesterol','meds_laxatives', 'ulcerative_colitis', 'kidney_disease','bladder_kidney','kidney_cancer','kidney_failure','celiac_disease','colonic_crohns','ileal_crohns','diverticulosis','inflammatory_bowel','irritable_bowel','gerd','peptic_ulcer','coagulation']
substrings_to_exclude = ['siblings','sister','mother','family','father','paternal','maternal','grandparent','self_past','gt1ya']
inclusion_mask = digest.columns.str.contains('|'.join(substrings_to_detect))
exclusion_mask = digest.columns.str.contains('|'.join(substrings_to_exclude))

self_and_no_family_exclusion = inclusion_mask & ~exclusion_mask
ignored_features = ~(np.insert(inclusion_mask[1:],0,False) & ~exclusion_mask)
digest_excluded_w_self_and_no_family = digest.loc[:, self_and_no_family_exclusion]  # Selects columns based on the mask

matching_indexes = digest_excluded_w_self_and_no_family[digest_excluded_w_self_and_no_family.isin(values).any(axis=1)].public_client_id.values
digest_remaining_w_self_and_no_family_excluded = digest.loc[~digest.public_client_id.isin(matching_indexes)].drop_duplicates(subset='public_client_id')

digest_remaining_w_self_and_no_family_excluded.loc[:, ignored_features]
result = digest_remaining_w_self_and_no_family_excluded['public_client_id']
print(len(bowel[bowel['public_client_id'].isin(result)])) #those remaining without self history of GI disease, kidney disease, or xenobiotic exclusion criteria


1561


In [28]:
(len(digest.public_client_id.unique()) #the total
 - 
 len(digest_remaining_w_self_and_no_family_excluded.public_client_id.unique()))/len(digest.public_client_id.unique())*100 # minus the remaining divided by the total gives % of excluded

38.50574712643678

In [29]:
len(lifestyle.public_client_id.unique())

5764

In [30]:
(len(digest.public_client_id.unique()) #the total
 - 
 len(digest_remaining_w_self_and_no_family_excluded.public_client_id.unique()))

1206

In [31]:
biodf['age'].unique()

array([54., 52., 48., 61., 31., 49., 30., 55., 68., 26., 34., 56., 42.,
       65., 67., 47., 27., 25., 33., 58., 32., 35., 46., 63., 60., 38.,
       62., 53., 29., 39., 37., 69., 41., 59., 51., 28., 40., 72., 66.,
       50., 19., 45., 43., 79., 44., 64., 24., 57., 70., 71., 36., 23.,
       21., 74., 75., 76., 78., 73., 77., 80., 22., 20., 87., 83., 82.,
       81., 89., 18., 84., 88.,  9., 86.])

In [32]:
biodf['age'].max() # the oldest in the cohort

89.0

In [34]:
#get anxiety and depression subcohorts
health_hist = adi.get_snapshot('assessments_health_history_new')
health_hist = health_hist.drop_duplicates(subset='public_client_id')


depression = pd.merge(lifestyle,health_hist)
pci = depression.filter(regex='public_client_id')
anxiety = depression.filter(regex='anxiety')
depression = depression.filter(regex='depression')

depression = pd.concat([pci, depression], axis = 1)                        
#depression = pd.merge(stress, depression)
depression = pd.merge(bowel, depression)
depression = pd.merge(biodf, depression)
depression = pd.merge(bmi, depression)
depression = pd.merge(eGFR, depression)
depression = pd.merge(blood, depression)
depression = pd.merge(anc, depression)


anxiety = pd.concat([pci, anxiety], axis = 1)                        
#depression = pd.merge(stress, anxiety)
anxiety = pd.merge(bmi, anxiety)
anxiety = pd.merge(biodf, anxiety)
anxiety = pd.merge(bowel, anxiety)
anxiety = pd.merge(eGFR, anxiety)
anxiety = pd.merge(blood, anxiety)
anxiety = pd.merge(anc, anxiety)

depression.bowel = depression.bowel.replace([1,2,3,4], ['Constipation','Low\nNormal','High\nNormal','Diarrhea'])
anxiety.bowel = anxiety.bowel.replace([1,2,3,4], ['Constipation','Low\nNormal','High\nNormal','Diarrhea'])

depression = depression.dropna(axis = 1, how = 'all')
depression = depression.drop(['PC4','PC5'], axis = 1) # only keep the first three principal components of genetic ancestry
depression.set_index('public_client_id').to_csv('depression.csv')
depression
#depression results


anxiety = anxiety.dropna(axis = 1, how = 'all')
anxiety = anxiety.drop(['PC4','PC5'], axis = 1)
anxiety.set_index('public_client_id').to_csv('anxiety.csv')
anxiety
# anxiety results
# 2096 participants

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.11/site-packages/arivale_data_interface/snapshots_interface.py:271: DeprecationWarning: `assessments_health_history_new` is slated for deprecation and will be part of `assessments` with raw data will be available in `assessments_raw`
  check_deprecation(name) # check if requested snapshot is to be deprecated


,public_client_id,PC1,PC2,PC3,CRP HIGH SENSITIVITY,LDL-CHOL CALCULATION,GLYCOHEMOGLOBIN A1C,eGFR,bowel,sex,age,BMI_CALC,assessment:health-history:mental_health_anxiety_disorder:self_current,assessment:health-history:mental_health_anxiety_disorder:self_past,assessment:health-history:mental_health_anxiety_disorder:self_past_year,assessment:health-history:mental_health_anxiety_disorder:self_past_gt1ya,assessment:health-history:mental_health_anxiety_disorder:mother,assessment:health-history:mental_health_anxiety_disorder:father,assessment:health-history:mental_health_anxiety_disorder:siblings,assessment:health-history:mental_health_anxiety_disorder:maternal,assessment:health-history:mental_health_anxiety_disorder:paternal,assessment:health-history:mental_health_anxiety_disorder:family,assessment:productivity:recent-ailments:anxiety
0,01001298,0.006588,-0.002195,-0.004136,0.60,187.0,5.6,99.024207,High\nNormal,F,61.0,26.622485,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,NaN
1,01001467,-0.024357,0.065986,-0.013913,0.56,117.0,5.2,100.959522,Low\nNormal,F,31.0,18.479394,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,NaN
2,01001548,0.006395,-0.002255,-0.003103,1.40,135.0,5.8,105.411124,High\nNormal,M,49.0,36.119222,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,NaN
3,01001621,0.006223,-0.001662,-0.003846,1.92,114.0,5.6,69.439409,Low\nNormal,F,54.0,23.294675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FALSE
4,01001661,-0.050232,-0.020917,-0.015837,1.08,84.0,5.6,120.947576,High\nNormal,F,30.0,21.031478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2091,HX980884,0.006184,-0.001006,-0.004071,2.74,93.0,5.0,119.020348,Low\nNormal,F,32.0,26.422571,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,NaN
2092,HX981293,0.002623,0.004474,0.008720,3.46,126.0,5.6,107.698203,High\nNormal,M,45.0,24.806553,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,NaN
2093,ZU273983,0.006136,-0.001745,-0.005167,11.92,65.0,5.0,118.179626,Low\nNormal,F,32.0,30.984144,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,NaN
2094,ZU612255,-0.026077,0.087718,-0.016342,8.28,146.0,5.4,70.256126,Low\nNormal,F,35.0,35.177122,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,NaN


In [36]:
# you can comment and uncomment the lines below to merge and create the overall dataframe from the metabolomics, proteomics, or gut microbiome snapshots:
output_df = pd.merge(bowel,biodf, on =['public_client_id'])
output_df = pd.merge(output_df, bmi, on=['public_client_id'])
output_df = pd.merge(output_df, eGFR, on=['public_client_id'])
output_df = pd.merge(output_df, anc, on=['public_client_id'])
#output_df = pd.merge(output_df, meds)
output_df = pd.merge(output_df, blood, on=['public_client_id'])
#output_df = pd.merge(output_df, rawmet, on=['public_client_id'])
#output_df = pd.merge(output_df,dash)
#output_df = pd.merge(output_df,taxa)
output_df = output_df.sort_values('public_client_id',ascending=True)
output_df = output_df.drop(['PC4','PC5'], axis = 1)
larger_cohort = output_df
nonPTR_df = output_df[output_df['public_client_id'].isin(result)]
nonfiltered_cohort = output_df
nonPTR_df = nonPTR_df.dropna()
nonPTR_df

,public_client_id,bowel,sex,age,BMI_CALC,eGFR,PC1,PC2,PC3,CRP HIGH SENSITIVITY,LDL-CHOL CALCULATION,GLYCOHEMOGLOBIN A1C
965,01001467,2,F,31.0,18.479394,100.959522,-0.024357,0.065986,-0.013913,0.56,117.0,5.2
1937,01001548,3,M,49.0,36.119222,105.411124,0.006395,-0.002255,-0.003103,1.40,135.0,5.8
966,01001621,2,F,54.0,23.294675,69.439409,0.006223,-0.001662,-0.003846,1.92,114.0,5.6
2592,01001661,3,F,30.0,21.031478,120.947576,-0.050232,-0.020917,-0.015837,1.08,84.0,5.6
968,01002192,3,M,26.0,32.711020,125.172870,0.007168,-0.002011,-0.004636,1.68,132.0,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...
943,HX927178,2,F,34.0,22.676409,103.744093,0.003632,0.003475,0.008164,8.51,133.0,5.2
18,HX980884,2,F,32.0,26.422571,119.020348,0.006184,-0.001006,-0.004071,2.74,93.0,5.0
17,HX981293,3,M,45.0,24.806553,107.698203,0.002623,0.004474,0.008720,3.46,126.0,5.6
1494,ZU273983,2,F,32.0,30.984144,118.179626,0.006136,-0.001745,-0.005167,11.92,65.0,5.0


In [38]:
# assemble the cohort for the ordinal POLR analysis downstream that will analyze survey questionnaire
# data on health, diet, and lifestyle to deduce important features related to BMF:
ordinal = digest
ordinal = pd.merge(ordinal, stress)
ordinal = pd.merge(nonPTR_df,ordinal)
ordinal = ordinal.rename(columns={'assessment:lifestyle:moderate_activity:enum':'exercise',
                               'assessment:digestion:bowel-movement-ease:enum': 'bowel_ease',
                               'assessment:digestion:laxatives:enum':'laxatives',
                                'assessment:diet:freq_meat:int':'freq_meat',
                                'assessment:diet:freq_processed_meat:int':'freq_procmeat',
                                'assessment:diet:freq_cruciferous_vegetables:int':'leafy_green_veg',
                                'assessment:diet:freq_cooked_green_vegetables:int':'cooked_green_veg',
                                'assessment:diet:freq_cooked_root_vegetables:int':'cooked_root_veg',
                                'assessment:pss-four-item-result:perceived_stress':'stress',
                                'assessment:diet:freq_fresh_fruit:int':'fresh_fruit_frequency',
                                'assessment:lifestyle:fruits:enum':'daily_fruit',
                                'stress_score':'pss',
                                'assessment:digestion:abdominal-pain:enum':'ab_pain',
                                'assessment:health-history:diabetes_adult:self_current':'diabetes'
                               })
ordinal.bowel = ordinal.bowel.replace([1,2,3,4], ['Constipation','Low\nNormal','High\nNormal','Diarrhea'])

# drop unnecessary columns:
ordinal = ordinal[ordinal.columns.drop(list(ordinal.filter(regex='journey')))]
ordinal = ordinal[ordinal.columns.drop(list(ordinal.filter(regex='_y')))]
ordinal = ordinal[ordinal.columns.drop(list(ordinal.filter(regex='_x')))]
ordinal = ordinal[ordinal.columns.drop(list(ordinal.filter(regex='is_remapped')))]
ordinal = ordinal[ordinal.columns.drop(list(ordinal.filter(regex='vendor')))]

nonna_counts=(~ordinal.isna()).sum() # sum up times any columns have non-NA values
ordinal = ordinal.loc[:, nonna_counts/ordinal.shape[0] >= 0.90] # of the fraction of non-missing values must be greater than 90%
ordinal_counts=(ordinal == 'FALSE').sum() # sum up times any columns have T/F values
ordinal = ordinal.loc[:, ordinal_counts/ordinal.shape[0] <= 0.90] #the fraction of FALSE values must be no greater than 90%
ordinal = ordinal[ordinal != 'FALSE']
ordinal = ordinal[ordinal != 'TRUE']
#Remove yes/no binary questions--they do not work with POLR:
df = ordinal
df = df.loc[:, ~(df == '(0) No').any()]
df = df.loc[:, ~(df == '(1) No').any()]
df = df.loc[:, ~(df == '(2) No').any()]
df = df.loc[:, ~(df == '(0) Yes').any()]
df = df.loc[:, ~(df == '(1) Yes').any()]
df = df.loc[:, ~(df == '(2) Yes').any()]
df = df.loc[:, ~(df == 'No').any()]
df = df.loc[:, ~(df == 'Yes').any()]
ordinal = df

ordinal = ordinal.dropna(axis = 1, how = 'all')
ordinal = ordinal.drop_duplicates('public_client_id')
ordinal = ordinal.drop(columns=['response_count','response_pct'])
ordinal.set_index('public_client_id').to_csv('ordinal_questions.csv')
ordinal # 1420 participants


,public_client_id,bowel,sex,age,BMI_CALC,eGFR,PC1,PC2,PC3,CRP HIGH SENSITIVITY,LDL-CHOL CALCULATION,GLYCOHEMOGLOBIN A1C,days_in_program,days_since_first_call,days_since_first_draw,month,weekday,season,ab_pain,assessment:digestion:acid_reflux:enum,assessment:digestion:bloating:enum,assessment:digestion:bowel-movement-completion:enum,bowel_ease,assessment:digestion:bowel-movements:enum,assessment:digestion:diarrhea:enum,assessment:digestion:gas:enum,assessment:digestion:high-appetite:enum,laxatives,assessment:digestion:medications:enum,assessment:digestion:nausea:enum,assessment:digestion:poor_or_lack_of_appetite:enum,assessment:digestion:stress-eating:enum,assessment:digestion:supplements:enum,assessment:digestion:vomiting:enum,assessment:lifestyle:achohol_type:enum,assessment:lifestyle:alcohol_days_a_week:enum,assessment:lifestyle:breakfast:enum,assessment:lifestyle:cruciferous_vegetables:enum,assessment:lifestyle:snack:enum,daily_fruit,assessment:lifestyle:vegetables:enum,assessment:lifestyle:sugary_drinks:enum,assessment:lifestyle:water:enum,assessment:lifestyle:alcohol_drinks_a_day:enum,assessment:lifestyle:diet:enum,assessment:lifestyle:grains:enum,assessment:satisfaction::enum,assessment:satisfaction:cheerful_effect:enum,assessment:satisfaction:dislike_way_i_am:enum,assessment:satisfaction:find_beauty:enum,assessment:satisfaction:find_time:enum,assessment:satisfaction:fun_with_people:enum,assessment:satisfaction:gap:enum,assessment:satisfaction:good_influence:enum,assessment:satisfaction:have_energy:enum,assessment:satisfaction:intense_interest_in_others:enum,assessment:satisfaction:involved:enum,assessment:satisfaction:joy_and_elation:enum,assessment:satisfaction:laugh_a_lot:enum,assessment:satisfaction:life_is_good:enum,assessment:satisfaction:life_is_rewarding:enum,assessment:satisfaction:make_decisions:enum,assessment:satisfaction:mentally_alert:enum,assessment:satisfaction:most_things_amusing:enum,assessment:satisfaction:no_happy_memories:enum,assessment:satisfaction:not_attractive:enum,assessment:satisfaction:not_healthy:enum,assessment:satisfaction:not_in_control:enum,assessment:satisfaction:not_optimistic:enum,assessment:satisfaction:purpose:enum,assessment:satisfaction:rarely_rested:enum,assessment:satisfaction:take_on_anything:enum,assessment:satisfaction:very_happy:enum,assessment:satisfaction:well_satisfied:enum,assessment:satisfaction:world_is_a_bad_place:enum,assessment:joyful-moments:feel_empowered:enum,assessment:joyful-moments:feel_lost_control:enum,assessment:joyful-moments:feel_rundown:enum,assessment:joyful-moments:healthy_life:enum,assessment:joyful-moments:lose_focus:enum,assessment:joyful-moments:right_track:enum,assessment:joyful-moments:satisfied:enum,assessment:lifestyle:childer:enum,assessment:lifestyle:grandchildren:enum,assessment:lifestyle:live_with:enum,exercise,assessment:lifestyle:moderate_activity_duration:enum,assessment:lifestyle:relationship_status:enum,assessment:lifestyle:time_seated:enum,assessment:lifestyle:vigorous_activity:enum,assessment:lifestyle:vigorous_activity_duration:enum,assessment:personality-result:agreeableness,assessment:personality-result:conscientiousness,assessment:personality-result:extraversion,assessment:personality-result:neuroticism,assessment:personality-result:openness,stress,assessment:satisfaction-result:happiness,assessment:personality:attention_to_detail:enum,assessment:personality:background:enum,assessment:personality:bad_imagination:enum,assessment:personality:center_of_attention:enum,assessment:personality:change_mood:enum,assessment:personality:comfortable_with_people:enum,assessment:personality:difficult_words:enum,assessment:personality:difficulty_abstract_ideas:enum,assessment:personality:disinterest_abstract_ideas:enum,assessment:personality:disinterest_peoples_problems:enum,assessment:personality:dislike_attention:enum,assessment:personality:do_chores:enum,assessment:personality:do_not_talk:enum,assessment:personality:easily_disturbed:enum,ass

In [ ]:
#Save final dataframes
#overall cohort:
nonPTR_df.to_csv('nonPTR_cohort.csv', index = False)

# metabolomics cohort:
metabolomics = pd.merge(nonPTR_df,rawmet, on=['public_client_id'])
metabolomics = metabolomics.drop_duplicates(subset='public_client_id')
metabolomics.set_index('public_client_id').to_csv('metabolomics.csv')
metabolomics.set_index('public_client_id').iloc[:,11:].transpose().to_csv('metabolomics_count.csv')

# gut microbiome cohort:
gutmicrobiome = pd.merge(nonPTR_df,dash, on=['public_client_id'])
gutmicrobiome = pd.merge(gutmicrobiome,taxa, on=['public_client_id'])
gutmicrobiome = gutmicrobiome.drop_duplicates(subset='public_client_id')
gutmicrobiome.set_index('public_client_id').to_csv('gutmicrobiome.csv')
gutmicrobiome.set_index('public_client_id').iloc[:,12:].transpose().to_csv('gutmicrobiome_count.csv')

gutmicrobiome_plotting = pd.merge(nonPTR_df,dash, on=['public_client_id'])
gutmicrobiome_plotting = pd.merge(gutmicrobiome_plotting,clr_taxa, on=['public_client_id'])
gutmicrobiome_plotting = gutmicrobiome_plotting.drop_duplicates(subset='public_client_id')
gutmicrobiome_plotting.set_index('public_client_id').to_csv('gutmicrobiome_plotting.csv')
gutmicrobiome_plotting.set_index('public_client_id').iloc[:,12:].transpose().to_csv('gutmicrobiome_plotting_count.csv')

# proteomics cohort:
proteomics = pd.merge(nonPTR_df,pro, on=['public_client_id'])
proteomics = proteomics.drop_duplicates(subset='public_client_id')
proteomics = proteomics.drop(['days_in_program','days_since_first_call','days_since_first_draw'],axis=1)
proteomics.set_index('public_client_id').to_csv('proteomics.csv')
proteomics.set_index('public_client_id').iloc[:,11:].transpose().to_csv('proteomics_count.csv')

# clinical laboratory chemistries cohort:
chemistries = pd.merge(nonPTR_df.iloc[:,:13],chemdf.drop(['LDL-CHOL CALCULATION', 'CRP HIGH SENSITIVITY', 'GLYCOHEMOGLOBIN A1C'], axis=1), on=['public_client_id'])
chemistries = chemistries.drop_duplicates(subset='public_client_id')
chemistries.set_index('public_client_id').to_csv('chemistries.csv')
chemistries.set_index('public_client_id').iloc[:,11:].transpose().to_csv('chemistries_count.csv')

In [24]:
# Additional operations and information
# get demographics data
lifestyle['assessment:vitals:race:enum'].value_counts()

assessment:vitals:race:enum
(0) White                                        2562
(2) Asian                                         262
(5) Other                                         144
(1) Black or African-American                      98
(3) Native Hawaiian or other Pacific Islander      20
(4) American Indian or Alaska Native                9
Name: count, dtype: int64

In [25]:
lifestyle['assessment:vitals:race:enum'].dropna().shape[0]

3095

In [26]:
lifestyle['assessment:vitals:ethnicity:enum'].value_counts()

assessment:vitals:ethnicity:enum
(1) Non-Hispanic    2897
(0) Hispanic         198
Name: count, dtype: int64

In [27]:
hispanic = lifestyle.filter(items = ['assessment:vitals:race:enum','assessment:vitals:ethnicity:enum'])
hispanic[(hispanic['assessment:vitals:ethnicity:enum']=='(0) Hispanic') & (hispanic['assessment:vitals:race:enum']!='(0) White')].value_counts()

assessment:vitals:race:enum                    assessment:vitals:ethnicity:enum
(5) Other                                      (0) Hispanic                        75
(1) Black or African-American                  (0) Hispanic                         7
(2) Asian                                      (0) Hispanic                         2
(3) Native Hawaiian or other Pacific Islander  (0) Hispanic                         2
(4) American Indian or Alaska Native           (0) Hispanic                         2
Name: count, dtype: int64

In [28]:
biodf[biodf['sex'] == 'M'] # number of males is 2140

,sex,age,public_client_id
0,M,54.0,01000552
5,M,49.0,01001548
10,M,26.0,01002192
18,M,47.0,01003555
19,M,27.0,01003662
...,...,...,...
6119,M,78.0,HX978690
6121,M,45.0,HX981293
6122,M,69.0,HX993544
6125,M,39.0,HX999862


In [29]:
biodf[biodf['sex'] == 'F'] # number of females is 3993

,sex,age,public_client_id
1,F,52.0,01000978
2,F,48.0,01001181
3,F,61.0,01001298
4,F,31.0,01001467
6,F,54.0,01001621
...,...,...,...
6128,F,44.0,QIRL
6129,F,32.0,ZU273983
6130,F,35.0,ZU612255
6131,F,37.0,ZU621944


In [30]:
biodf

,sex,age,public_client_id
0,M,54.0,01000552
1,F,52.0,01000978
2,F,48.0,01001181
3,F,61.0,01001298
4,F,31.0,01001467
...,...,...,...
6128,F,44.0,QIRL
6129,F,32.0,ZU273983
6130,F,35.0,ZU612255
6131,F,37.0,ZU621944


In [31]:
#get average and stdev of various factors in cells below
print(3993/6133)

0.651067992825697


In [32]:
bmi.BMI_CALC.mean()

27.151684222650097

In [33]:
bmi.BMI_CALC.std()

5.89026033228982

In [34]:
biodf.age.mean()

46.3634436654166

In [35]:
biodf.age.std()

12.960027386597147

In [36]:
eGFR.eGFR.mean()

89.06766883034909

In [37]:
eGFR.eGFR.std()

20.203367860462112

In [38]:
blood['CRP HIGH SENSITIVITY'].mean()

2.395680606806068

In [39]:
blood['CRP HIGH SENSITIVITY'].std()

4.758290949918183

In [40]:
blood['LDL-CHOL CALCULATION'].mean()

114.17275541795665

In [41]:
blood['LDL-CHOL CALCULATION'].std()

33.77058782317881

In [42]:
blood['GLYCOHEMOGLOBIN A1C'].mean()

5.493949958982773

In [43]:
blood['GLYCOHEMOGLOBIN A1C'].std()

0.5742922554097244

In [73]:
result.count()

151

In [74]:
(lifestyle.public_client_id.count())

5764

In [76]:
#Testing
laxmed = adi.get_snapshot('assessments')
laxmed = laxmed[['public_client_id','days_in_program','assessment:digestion:laxatives:enum']]
cholmed = adi.get_snapshot('assessments_medications')
cholmed = cholmed[['public_client_id','days_in_program','meds_cholesterol','meds_blood_pressure']]
laxmed = laxmed.rename(columns={'assessment:digestion:laxatives:enum':'meds_laxatives'})
meds = pd.merge(laxmed,cholmed)
meds

#remove those who report (Yes) below from the regression cohort.
hh = adi.get_snapshot('assessments_health_history_new')
dig = adi.get_snapshot('assessments_digestive_health')
#hh2 = adi.get_snapshot('assessments_health_history_old')


#Retrieve miscellaneous digestive health indicators: 
lifestyle = adi.get_snapshot('assessments') 
lifestyle = lifestyle.drop_duplicates(subset='public_client_id')
lifestyle = lifestyle.sort_values('days_in_program',ascending=True)

#diabetes = hh[['public_client_id','assessment:health-history:diabetes_adult:self_current']]

meds = meds[['public_client_id','meds_laxatives','meds_cholesterol','meds_blood_pressure']]

#common_columns = list(set(hh.columns).intersection(hh2.columns))
digest = pd.merge(hh,dig, on = list(['vendor','days_in_program','days_since_first_call','days_since_first_draw','month','weekday','season',"public_client_id"]))
#digest = pd.merge(digest,dig, on = 'public_client_id')
#digest = pd.merge(digest, diabetes, on = 'public_client_id')
digest = pd.merge(digest, meds, on = "public_client_id")

common_columns = list(set(digest.columns).intersection(lifestyle.columns))
digest = pd.merge(digest, lifestyle, on=common_columns)

# create a list of values to check
values = ['Yes', 'Y', 'True','TRUE',True, 1
          ,'(3) Once per week or less', '(1) Daily','(2) Several times per week', '(2) Currently'
         ]

kidney_substrings_to_detect = ['public_client_id','kidney_disease','bladder_kidney','kidney_cancer','kidney_failure']
gi_substrings_to_detect = ['public_client_id','ulcerative_colitis', 'celiac_disease', 'colonic_crohns','ileal_crohns','diverticulosis','inflammatory_bowel','irritable_bowel','gerd','peptic_ulcer']

substrings_to_exclude = ['siblings','sister','mother','family','father','paternal','maternal','grandparent','self_past','gt1ya']

inclusion_mask = digest.columns.str.contains('|'.join(gi_substrings_to_detect))
exclusion_mask = digest.columns.str.contains('|'.join(substrings_to_exclude))

final_mask = inclusion_mask & ~exclusion_mask
alternative_mask = ~(np.insert(inclusion_mask[1:],0,False) & ~exclusion_mask)
mask_df = digest.loc[:, final_mask]  # Selects columns based on the mask

matching_indexes = mask_df[mask_df.isin(values).any(axis=1)].public_client_id.values

digest = digest.loc[digest.public_client_id.isin(matching_indexes)].drop_duplicates(subset='public_client_id')

digest = digest.loc[:, alternative_mask]

result = digest['public_client_id']
result


/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.11/site-packages/arivale_data_interface/snapshots_interface.py:271: DeprecationWarning: `assessments_medications` is slated for deprecation and will be part of `assessments` with raw data will be available in `assessments_raw`
  check_deprecation(name) # check if requested snapshot is to be deprecated
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.11/site-packages/arivale_data_interface/snapshots_interface.py:271: DeprecationWarning: `assessments_health_history_new` is slated for deprecation and will be part of `assessments` with raw data will be available in `assessments_raw`
  check_deprecation(name) # check if requested snapshot is to be deprecated
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.11/site-packages/arivale_data_interface/snapshots_interface.py:271: DeprecationWarning: `assessments_digestive_health` is slated for deprecation and will be part of `assessments` with raw data will be available in `assessm

7       01001798
16      01003662
22      01004551
29      01005100
30      01005935
          ...   
3742    HX411182
3749    HX611763
3750    HX662917
3752    HX748976
3761        INEW
Name: public_client_id, Length: 445, dtype: object

In [77]:
result.count()

445

In [78]:
(lifestyle.public_client_id.count())

5764

In [79]:
print(151/5764)

0.026197085357390702


In [80]:
print(445/5764)

0.07720333102012492
